# 推薦模型

## 前言(通常-->廢話)
早餐去買三明治的時候，老闆娘總會好心的問說，要不要來杯紅茶。通常看在老闆娘的~~顏值~~誠意上，都會不爭氣的買了...推薦是針對原本客戶沒有預期到的結果，額外的跨售商品，或提示給用戶可能感興趣（喜歡）的東西（例如:買早餐時原本不會口渴，受到老闆娘的~~顏值~~影響才加購飲料的。）

推薦方法火熱了這麼多年是因為在商業上有很實際的目的：

1. 促進交易
2. 加速決策

簡單來說對商家來說，就是賺更多`$$`。對消費者來說可以從雜亂無章，一堆無用訊息裡面過濾有興趣的商品。在資訊過載的世界裡面，消費者無法一下子就能找到感興趣的東西。谷歌搜尋技法，在目的不明確的狀況下，沒辦法發揮功能。

在網路時代，商家通常能夠快速的蒐集到用戶的購買/瀏覽/蒐藏紀錄，讓蒐集到的資訊，透過模型建立有機會，找到相似（相關性高）的商品來推薦（推播）。建構在模型上，消費者體驗的整體流程可以稱之為系統，系統面結合的範圍較大不在本文討論範圍（系統架構面本人也不懂，只懂簡單的幾種模型）。這一系列的文章，希望能紀錄自己對幾種熱門模型的理解。

一般來說（就本人所知）有以下的算法框架
_______

## 算法框架

* 協同過濾法(Collaborative Filtering)
    - 相似度（最鄰近法-KNN)
        - ubcf, ibcf
    - 矩陣分解(Matrix Factorization)
        - explict (implicit) ALS
        - SVD, SVD++, SVDFeatures ...
        - Learning to rank (BSR,WARP...)
    - 深度學習
        - DNN
* 以內容最基礎(Content-based)

![framework](img/算法框架.png)
_____________________________

## 最鄰近法(相似度)
協同過濾法簡單說，就是透過群體的交易行為來推算，小王購買A,B,C的可能機會。透過群體關係，來推論誰和你的購買歷程相近...最大的特色是，不需要對商品/領域有專業知識即能有相當的準確度。其中最鄰近法(KNN)以考慮最相鄰的N人(物品)來計算，不算是一種[模型]，只是一種群眾給出的統計結果。沒有模型常見的訓練過程

1. 定義cost function,
2. 求得最佳參數
3. 評估模型結果

有以交易紀錄形成的資料如下，

| |趙 |錢 |孫 | 李|
|---|---|---|---|---|
|牙刷|null|V|null|V|
|牙膏|null|V|null|null|
|腳踏車|null|null|V|V|
|LED燈|V|null|null|V|
|澡盆|V|V|null|V|

以商品的角度來看，牙刷的交易紀錄`(0,1,0,1)`形成空間一組向量，可以計算與牙膏的相似度`(0,1,0,0)`。能建構所有商品的相似度表格

$$
cos(\theta) = \frac{r_{牙膏} \cdot r_{牙刷}'}{\lVert{r_{牙膏}}\lVert \lVert r_{牙刷}' \lVert}
$$

In [5]:
import numpy as np

def similarity(rating,kind='ubcf',eps=1e-9):
    if kind =='ubcf':
        sim = np.dot(rating,rating.T) + eps
    if kind =='ibcf':
        sim = np.dot(rating.T,rating) + eps
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return sim/norms/norms.T

rating_t = np.array([
    [0,1,0,1],
    [0,1,0,0],
    [0,0,1,1],
    [1,0,0,1],
    [1,1,0,1]
  ])
rating = rating_t.T
sim_i = similarity(rating,kind='ibcf')
sim_i.astype(np.float16)

array([[ 1.        ,  0.70703125,  0.5       ,  0.5       ,  0.81640625],
       [ 0.70703125,  1.        ,  0.        ,  0.        ,  0.57714844],
       [ 0.5       ,  0.        ,  1.        ,  0.5       ,  0.40820312],
       [ 0.5       ,  0.        ,  0.5       ,  1.        ,  0.81640625],
       [ 0.81640625,  0.57714844,  0.40820312,  0.81640625,  1.        ]], dtype=float16)

有相似度矩陣後，就能針對每個用戶還沒買的商品作預測。

$$
\hat{r_{ui}} = \sum_{u'}Sim(u,u') \cdot r_{u'i}
$$
or 
$$
\hat{r_{ui}} = \sum_{i'} r_{ui'} \cdot Sim(i,i')
$$

In [6]:
score_ibcf = rating.dot(sim_i).astype(np.float16)
score_ibcf

array([[ 1.31640625,  0.57714844,  0.90820312,  1.81640625,  1.81640625],
       [ 2.5234375 ,  2.28515625,  0.90820312,  1.31640625,  2.39453125],
       [ 0.5       ,  0.        ,  1.        ,  0.5       ,  0.40820312],
       [ 2.81640625,  1.28417969,  2.40820312,  2.81640625,  3.04101562]], dtype=float16)

以物品為基礎

    - 牙膏澡盆相似度最高(0.816),牙膏牙刷相似度次高(0.7)
    - 推薦孫：牙刷(0.5), LED燈(0.5) 已買腳踏車
____________________    

## 推薦實例Sketchfab Demo

我學習推薦系算法，除了阿撒不魯的網站論文看了一大堆，後來咀嚼思考後，覺得大概有87%的想法，源碼精神是來自於[DataPique](http://blog.ethanrosenthal.com/)大大的網站（其他的可謂~~垃圾?~~超越小弟~~腦弱~~的理解)，這個大大除了把最重要的幾種模型詳細推導一遍，更有趣的是作者(ethan)更自己爬了3DCAD的資料，蒐集第一手的客戶資料情況。真正公司會在網站後蒐集到的資料，通常都是客戶的隱式行為...(所以用這組資料來練習是最好不過的!)

隱性資料(implicit data)是大部分人會在瀏覽網站的狀況，
1. 好棒棒的點讚，
2. 覺得很爛/很腦殘~~智缺~~/沒興趣，通常是不會有反應的（空值）。

對真實世界來講，通常沒有好棒棒的評分資料，給你好棒棒的評分（像是啥鬼MovieLens影評資料...)。老實說我在聽mixerbox,spotify,netflix不好聽(看)就下一首(片)或關掉了。只有聽到好棒棒的影視/音樂，才會點讚讚。

之後會模仿DataPique的作法，在這個[資料](https://github.com/ihongChen/PlayRecommendSystem/tree/master/rec-a-sketch)上利用KNN來實踐一次。工人智慧看看推薦結果如何...


### 資料馬殺雞
---最囉唆的就是資料清理了---

* 資料通常很稀疏，要用`scipy.sparse`下面的稀疏矩陣來建構資料。
    - [scipy lecture note](http://www.scipy-lectures.org/advanced/scipy_sparse/index.html)有教學說這咪一大堆的稀疏矩陣差異性(~~累死寶寶~~)。
* 馬殺雞函數放在`rec_helper.py`裡面
* 模型放在`KNNmodel.py`下的`KNNmodel`
    - KNNmodel 有機會可以再重構,目前還算堪用    
    

In [ ]:
!conda install tqdm

In [1]:
import numpy as np 
import pandas as pd
import csv
import sys

In [2]:
sys.path.append('../')

In [3]:
from KNNmodel import *
from rec_helper import *

In [4]:
df = pd.read_csv('../rec-a-sketch/model_likes_anon.psv',
                 sep='|',quotechar='\\',quoting=csv.QUOTE_MINIMAL)
df.head()

,modelname,mid,uid
0,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,7ac1b40648fff523d7220a5d07b04d9b
1,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,2b4ad286afe3369d39f1bb7aa2528bc7
2,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,1bf0993ebab175a896ac8003bed91b4b
3,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,6484211de8b9a023a7d9ab1641d22e7c
4,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,1109ee298494fbd192e27878432c718a


刪除部份重複資料

In [5]:
print(df.count())
df.drop_duplicates(inplace=True)
print(df.count())

modelname    632832
mid          632832
uid          632832
dtype: int64
modelname    632677
mid          632677
uid          632677
dtype: int64


移除喜歡次數過少(小於5)的用戶,或少於10人按過讚的3D圖

In [6]:
df = threshold_interaction(df,rowname='uid',colname='mid',row_min=5,col_min=10)

Starting interactions info
Number of rows: 62583
Number of cols: 28806
Sparsity: 0.04%
Ending interactions info
Number of rows: 13496
Number of columns: 13618
Sparsity: 0.25%


存成稀疏矩陣與代碼編號對應

In [7]:
inter,uid_to_idx,idx_to_uid,mid_to_idx,idx_to_mid=df_to_spmatrix(df,'uid','mid')

切切切資料

In [8]:
## train,test split
train,test, user_idxs = train_test_split(inter,split_count=1,fraction=0.2)

In [9]:
%time train_coo = train.tocoo(copy=True) # csr -> coo 代價超小!!驚驚!!

Wall time: 7.01 ms


### 建立KNN模型

In [10]:
model_u = KNNmodel(train,kind='ubcf')
model_u.jaccard_sim()
model_u.fit(topK=50,remove=True) # 取前50個最近鄰計算

similarity (jaccard) matrix built (ubcf), 
sparsity of similarity: 12.02 %
-- start building topK user array...


100%|███████████████████████████████| 13496/13496 [00:08<00:00, 1509.78users/s]


-- end building topK user array---
start building prediction rating...


100%|████████████████████████████████| 13496/13496 [00:44<00:00, 306.58users/s]


ubcf rating matrix built...

arrange purchased data in rating matrix...
remove=True


100%|██████████████████████████| 13432420/13432420 [00:13<00:00, 964517.72it/s]


numbers of rows : 13496
numbers of cols: 13618
sparsity of rating: 7.31 %
save into *.rating attribute...


In [12]:
model_i = KNNmodel(train,kind='ibcf')
model_i.jaccard_sim()
model_i.fit(topK=50,remove=True)

similarity (jaccard) matrix built (ibcf), 
sparsity of similarity: 53.09 %


100%|█████████████████████████████████| 13618/13618 [04:10<00:00, 54.29items/s]


ibcf rating matrix built...

arrange purchased data in rating matrix...
remove=True


100%|████████████████████████████| 8557784/8557784 [00:09<00:00, 938638.97it/s]


numbers of rows : 13496
numbers of cols: 13618
sparsity of rating: 4.66 %
save into *.rating attribute...


In [14]:
## popular
model_p = KNNmodel(train,kind='popular')
model_p.fit(topK=50,remove=True)

100%|██████████████████████████████████| 13496/13496 [00:02<00:00, 6074.41it/s]


popular rating matrix built...

arrange purchased data in rating matrix...
remove=True


100%|██████████████████████████████| 674800/674800 [00:00<00:00, 853418.49it/s]


numbers of rows : 13496
numbers of cols: 13618
sparsity of rating: 0.37 %
save into *.rating attribute...


# 猜你喜歡

這部分使用人眼來觀察一下，推薦出來的結果長的怎樣...

偷自[Data Pique](http://blog.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)

In [151]:
import requests
def get_thumbnails(sim, idx, idx_to_mid, N=10):
    row = sim[idx, :].A.ravel()
    thumbs = []
    for x in np.argsort(-row)[:N]:
        response = requests.get('https://sketchfab.com/i/models/{}'.format(idx_to_mid[x])).json()
        thumb = [x['url'] for x in response['thumbnails']['images'] if x['width'] == 200 and x['height']==200]
        if not thumb:
            print('no thumbnail')
        else:
            thumb = thumb[0]
        thumbs.append(thumb)
    return thumbs

In [152]:
item_sim = model_i.sim
thumbs = get_thumbnails(item_sim, 0, idx_to_mid)

no thumbnail
no thumbnail
no thumbnail
no thumbnail


In [65]:
thumbs_exists = [e for e in thumbs if e != []]

In [153]:
thumbs

[[],
 [],
 'https://media.sketchfab.com/urls/18d38b440a9647059983c0e064b54948/dist/thumbnails/4cf3dbf1096d4d299c6181ec8f67a149/200x200.jpeg',
 'https://media.sketchfab.com/urls/6234f9020abd4ac79ebcdf6e09feff38/dist/thumbnails/0779456376614b65ad57dc06d2904481/200x200.jpeg',
 [],
 [],
 'https://media.sketchfab.com/urls/0ad160d8515949169e2f09e912879a20/dist/thumbnails/6e85aeecf94c4ebb90b7127b7b9e819d/200x200.jpeg',
 'https://media.sketchfab.com/urls/82367f6a22644f6f88b462077611559e/dist/thumbnails/9041c2a16d43466aaf81ee145ed20d3f/200x200.jpeg',
 'https://media.sketchfab.com/urls/efe433deeeba4588bc089bba71fef9a5/dist/thumbnails/8bbd97ec0c3f481787694ca8d33211c4/200x200.jpeg',
 'https://media.sketchfab.com/urls/4883769e53794a4295f25e49993f04d7/dist/thumbnails/3624d44225144991ae69d7b3a63c2f54/200x200.jpeg']

In [154]:
display(HTML('<img src='+ '\"'+thumbs[2]+'\">'))

In [45]:
from IPython.display import display, HTML


In [156]:
thumb_html = '<img src='+ '\"'+thumbs[2]+'\">' + '<img src='+ '\"'+thumbs[3]+'\">' + '<img src='+ '\"'+thumbs[-2]+'\">'
display(HTML(thumb_html))

In [149]:
def display_item(thumbs,N=5):
    try: 
        thumb_html = '<img src='+ '\"'+thumbs[0]+'\">' 
    except TypeError:
        print('No thumbnail...origin')
        thumb_html= ""
    for url in thumbs[1:]:
        if url:
            thumb_html = thumb_html + '<img src='+ '\"'+  url + '\">'     
    return thumb_html

In [157]:
HTML(display_item(thumbs))

No thumbnail...origin


宅氣十足...顆顆!!

_____

# 評估

- 準確度
- 召回度

![看投影片吧](./img/評估方法.png)

In [12]:
uids = np.arange(0,train.shape[0])
predall_u = model_u.predict(uids,topN=10)
model_u.evaluate(predall_u,test,method='recall') # recall: 16.6 %

predall_p = model_p.predict(uids,topN=10)
model_p.evaluate(predall_p,test,method='recall') # recall:3.08 %


-------------
model: ubcf,
topN: 10
recall:16.19 %

-------------
model: popular,
topN: 10
recall:3.15 %


In [20]:
predall_i = model_i.predict(uids,topN=10)
model_i.evaluate(predall_i,test,method='recall') #recall: 12 %


-------------
model: ibcf,
topN: 10
recall:12.08 %


In [17]:
model_u.evaluate(predall_u,test,method='precision')
model_i.evaluate(predall_i,test,method='precision')
model_p.evaluate(predall_p,test,method='precision')


-------------
model: ubcf,
topN: 10
precision: 1.66 %

-------------
model: ibcf,
topN: 10
precision: 1.20 %

-------------
model: popular,
topN: 10
precision: 0.31 %


## 小結

### 最近鄰法(K nearest neighbor):

1. 考量兩兩相似度，然後針對用戶(物)最相似的K個人(物)推薦他們也買的商品。
2. 是一種基於偷看別人也買什麼的偷窺法
3. 僅由最鄰近方法能打敗熱門商品的推薦(base line)
4. 此方法必須計算兩兩之間pair-wise的相似度，在記憶體的消耗上非常巨大。計算量隨用戶(商品)成$N^2$的增加問題...
5. 改善方法須透過降維與近似近鄰法(approximate nearest neighbor)來達成，會放在下一篇來討論...Netflix/spotify...最經典的算法 - 矩陣分解法